# Imports

Q1: Entity schedule has "Port Aventura World" in   it

In [ ]:
import polars as pl
from polars import col
import polars.selectors as cs
import pyarrow as pa
import pandas as pd
import numpy as np

# Set the plotting backend to Plotly
pd.options.plotting.backend = "plotly"

from datetime import date

# Attendance

In [2]:
attendance = (
  pl.read_csv('data/attendance.csv').filter(col('FACILITY_NAME') == 'PortAventura World')
  .with_columns(col('USAGE_DATE').str.to_datetime("%m/%d/%Y"))
  .select('USAGE_DATE', 'attendance')
)

In [150]:
attendance.head()

USAGE_DATE,attendance
datetime[μs],i64
2018-06-01 00:00:00,46804
2018-06-02 00:00:00,57940
2018-06-03 00:00:00,44365
2018-06-04 00:00:00,37617
2018-06-05 00:00:00,32438


In [5]:
attendance.to_pandas().plot(x= 'USAGE_DATE', y='attendance')

In [6]:
attendance.filter(
  (col('USAGE_DATE') <= date(2020, 3, 13)).or_(col('USAGE_DATE') >= date(2021, 6, 19))
).groupby(col('USAGE_DATE').dt.weekday()).mean().sort('USAGE_DATE', descending=False)

C:\Users\cedom\AppData\Local\Temp\ipykernel_15132\1545333220.py:5: DeprecationWarning:

`groupby` is deprecated. It has been renamed to `group_by`.



USAGE_DATE,attendance
i8,f64
1,35889.966887
2,35179.397351
3,34161.306667
4,35441.28
5,38461.589404
6,49241.701987
7,40890.675497


In [7]:
attendance.filter(
  (col('USAGE_DATE') <= date(2020, 3, 13))
  .or_
  (col('USAGE_DATE') >= date(2021, 6, 19))
).groupby(col('USAGE_DATE').dt.weekday()).mean().sort('USAGE_DATE', descending=False).to_pandas().plot(x = 'USAGE_DATE', y= 'attendance')

C:\Users\cedom\AppData\Local\Temp\ipykernel_15132\2205245149.py:5: DeprecationWarning:

`groupby` is deprecated. It has been renamed to `group_by`.



# Link attraction-park

In [19]:
link = pl.read_csv('data/link_attraction_park.csv', separator=';')

In [20]:
link.head().to_numpy()

array([['Aeroplane Ride', 'Tivoli Gardens'],
       ['Bumper Cars', 'PortAventura World'],
       ['Bungee Jump', 'PortAventura World'],
       ['Circus Train', 'PortAventura World'],
       ['Crazy Bus', 'Tivoli Gardens']], dtype=object)

# Entity Schedule

In [157]:
link.head()

ATTRACTION,PARK
str,str
"""Aeroplane Ride…","""Tivoli Gardens…"
"""Bumper Cars""","""PortAventura W…"
"""Bungee Jump""","""PortAventura W…"
"""Circus Train""","""PortAventura W…"
"""Crazy Bus""","""Tivoli Gardens…"


In [27]:
pl.read_csv('data/entity_schedule.csv').filter(col('ENTITY_DESCRIPTION_SHORT') == 'PortAventura World').head(2)

REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE
str,str,str,str,str,str,str
null,"""PortAventura W…","""PARK""","""2018-07-17 08:…","""2018-07-17 09:…","""2018-07-18 07:…","""2018-07-17"""
null,"""PortAventura W…","""PARK""","""2019-02-28 08:…","""2019-02-28 09:…","""2019-03-01 09:…","""2019-02-28"""


In [41]:
schedule = (
  pl.read_csv('data/entity_schedule.csv')
  .join(link, left_on = 'ENTITY_DESCRIPTION_SHORT', right_on = 'ATTRACTION', how='left')
  .filter(col('PARK') == 'PortAventura World')
  .with_columns(cs.contains(('TIME', 'DATE')).str.to_datetime())
)

In [42]:
schedule.head()

REF_CLOSING_DESCRIPTION,ENTITY_DESCRIPTION_SHORT,ENTITY_TYPE,DEB_TIME,FIN_TIME,UPDATE_TIME,WORK_DATE,PARK
str,str,str,datetime[μs],datetime[μs],datetime[μs],datetime[μs],str
null,"""Dizzy Dropper""","""ATTR""",2022-04-07 08:30:00,2022-04-07 22:04:00,2022-04-08 08:00:30,2022-04-07 00:00:00,"""PortAventura W…"
"""Fermeture Réha…","""Kiddie Coaster…","""ATTR""",2018-02-21 23:59:00,2018-02-21 23:59:00,2018-02-22 08:36:23,2018-02-21 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Oz Theatre""","""ATTR""",2018-09-04 23:59:00,2018-09-04 23:59:00,2018-09-05 08:12:54,2018-09-04 00:00:00,"""PortAventura W…"
null,"""Free Fall""","""ATTR""",2020-01-04 08:11:00,2020-01-04 18:15:00,2020-01-05 07:52:42,2020-01-04 00:00:00,"""PortAventura W…"
"""Fermeture Opér…","""Oz Theatre""","""ATTR""",2018-06-24 23:59:00,2018-06-24 23:59:00,2018-06-25 08:03:37,2018-06-24 00:00:00,"""PortAventura W…"


# Waiting times

In [34]:
wait = (
  pl.read_csv('data/waiting_times.csv')
  .join(link, left_on = 'ENTITY_DESCRIPTION_SHORT', right_on = 'ATTRACTION', how='left')
  .filter(col('PARK') == 'PortAventura World')
)

In [65]:
wait.head()

WORK_DATE,DEB_TIME,DEB_TIME_HOUR,FIN_TIME,ENTITY_DESCRIPTION_SHORT,WAIT_TIME_MAX,NB_UNITS,GUEST_CARRIED,CAPACITY,ADJUST_CAPACITY,OPEN_TIME,UP_TIME,DOWNTIME,NB_MAX_UNIT,PARK
str,str,i64,str,str,i64,f64,f64,f64,f64,i64,i64,i64,i64,str
"""1/1/2018""","""00:00.0""",21,"""15:00.0""","""Roller Coaster…",0,2.0,0.0,0.0,0.0,0,0,0,2,"""PortAventura W…"
"""1/1/2018""","""30:00.0""",19,"""45:00.0""","""Bumper Cars""",5,18.0,148.0,254.749,254.75,15,15,0,18,"""PortAventura W…"
"""1/1/2018""","""30:00.0""",22,"""45:00.0""","""Rapids Ride""",0,1.0,0.0,0.0,0.0,0,0,0,2,"""PortAventura W…"
"""1/1/2018""","""45:00.0""",12,"""00:00.0""","""Crazy Dance""",5,1.0,46.0,250.001,250.0,15,15,0,1,"""PortAventura W…"
"""1/1/2018""","""15:00.0""",18,"""30:00.0""","""Free Fall""",50,3.0,0.0,0.0,0.0,0,0,0,3,"""PortAventura W…"


In [78]:
wt = (
  wait
  .with_columns(
    pl.col("WORK_DATE").str.strptime(pl.Date, "%m/%d/%Y").cast(pl.Datetime).alias("WORK_DATE_DT"))
  .with_columns(
    (pl.col("WORK_DATE_DT") + pl.duration(hours = "DEB_TIME_HOUR") + pl.duration(minutes = col("DEB_TIME").str.split(":").list.get(0).cast(pl.Int32))).alias("DEB_DATETIME"),
    (pl.col("WORK_DATE_DT") + pl.duration(hours = "DEB_TIME_HOUR") + pl.duration(minutes = col("FIN_TIME").str.split(":").list.get(0).cast(pl.Int32))).alias("FIN_DATETIME"),
    )
  .drop('WORK_DATE_DT', 'WORK_DATE', 'DEB_TIME', 'FIN_TIME', 'DEB_TIME_HOUR')
)

In [84]:
wt.select(col('ENTITY_DESCRIPTION_SHORT').unique())

ENTITY_DESCRIPTION_SHORT
str
"""Roller Coaster…"
"""Inverted Coast…"
"""Kiddie Coaster…"
"""Crazy Dance"""
"""Haunted House"""
"""Free Fall"""
"""Giant Wheel"""
"""Oz Theatre"""
"""Rapids Ride"""


In [156]:
top_6_rides = (
  wt
  .group_by(col('ENTITY_DESCRIPTION_SHORT'))
  .agg(col('WAIT_TIME_MAX').mean().suffix('_mean'))
).sort('WAIT_TIME_MAX_mean').tail(6).to_numpy()

[sublist[0] for sublist in top_6_rides]

C:\Users\cedom\AppData\Local\Temp\ipykernel_15132\3737640886.py:4: DeprecationWarning:

`suffix` is deprecated. It has been moved to `name.suffix`.



['Spinning Coaster',
 'Go-Karts',
 'Free Fall',
 'Swing Ride',
 'Giant Wheel',
 'Spiral Slide']

In [154]:
(
  wt
  .filter((col('DEB_DATETIME') <= date(2020, 3, 13)).or_(col('DEB_DATETIME') >= date(2021, 6, 19)))   # We take out COVID TIME
  .group_by(col('ENTITY_DESCRIPTION_SHORT').alias('RIDE_NAME'), col('DEB_DATETIME').dt.weekday().alias('weekday'), (col('DEB_DATETIME').dt.hour() + col('DEB_DATETIME').dt.minute() / 60).alias('hour_minute'))
  .agg(col('WAIT_TIME_MAX').mean().suffix('_mean'))
  .filter(col('RIDE_NAME').is_in(['Spinning Coaster', 'Go-Karts', 'Free Fall', 'Swing Ride', 'Giant Wheel', 'Spiral Slide']))
).sort(['weekday', 'hour_minute'], descending=[False, False]).to_pandas().plot(
  x= 'hour_minute',
  y='WAIT_TIME_MAX_mean',
  color='weekday',
  facet_col="RIDE_NAME",
  facet_col_wrap = 3,
  title = 'Average wait time (m) at attraction X throughout the day', width=2000, height=1000) 

C:\Users\cedom\AppData\Local\Temp\ipykernel_15132\371080169.py:5: DeprecationWarning:

`suffix` is deprecated. It has been moved to `name.suffix`.



In [155]:
(
  wt
  .filter((col('DEB_DATETIME') <= date(2020, 3, 13)).or_(col('DEB_DATETIME') >= date(2021, 6, 19)))   # We take out COVID TIME
  .group_by(col('ENTITY_DESCRIPTION_SHORT').alias('RIDE_NAME'), col('DEB_DATETIME').dt.weekday().alias('weekday'), (col('DEB_DATETIME').dt.hour() + col('DEB_DATETIME').dt.minute() / 60).alias('hour_minute'))
  .agg(col('WAIT_TIME_MAX').mean().suffix('_mean'))
  .filter(col('RIDE_NAME').is_in(['Spinning Coaster', 'Go-Karts', 'Free Fall', 'Swing Ride', 'Giant Wheel', 'Spiral Slide']))
).sort(['weekday', 'hour_minute'], descending=[False, False]).to_pandas()

C:\Users\cedom\AppData\Local\Temp\ipykernel_15132\998782904.py:5: DeprecationWarning:

`suffix` is deprecated. It has been moved to `name.suffix`.



,RIDE_NAME,weekday,hour_minute,WAIT_TIME_MAX_mean
0,Spiral Slide,1,9.00,59.097222
1,Free Fall,1,9.00,4.452055
2,Go-Karts,1,9.00,6.712329
3,Swing Ride,1,9.00,16.849315
4,Giant Wheel,1,9.00,1.849315
...,...,...,...,...
2347,Giant Wheel,7,22.75,3.402778
2348,Free Fall,7,22.75,0.000000
2349,Spinning Coaster,7,22.75,1.944444
2350,Go-Karts,7,22.75,2.361111
